In [1]:
import os
import pandas as pd
import numpy as np
import subprocess
import re
import pickle
from make import make_dir,recover_make,all_,comp_,clang_flags,make_proc,make_routine,mopt_routine
from make import opt_bc,opt_routine,targ_routine,phony_routine,clean_routine,reg_name,select_groups
opt_ = ["-O%s" %i for i in [0,1,2,3,'s','z']]

clang_ =["-O%s" %i for i in [0,1,2,3,'fast','s','z','g','4']]
all_divide = r'all\s*\:\s*'
w_out_space_divide = r'(?<!\\)\s+'
name_divide = r'.+\s*\:\s+.*\.[cpx]{1,3}.*'
cf_reg = r'\.[cpCP]{1,3}'

frame = pd.DataFrame(columns=opt_,index=clang_)
absdir = "./../../../../Unix_Source/"
initdir = os.getcwd()+'/'
largedir = absdir+'unix_cmp'



In [2]:
def estb_src(abs_ls):
#     print(os.listdir())
    for res in os.listdir('result/'):
        tmp = pickle.load(open('result/'+res,'rb'))
        abs_ls = abs_ls.difference(tmp)
    return abs_ls

def log_file(lines,name=initdir+'complex.comp'):
    with open(name,'a+') as file:
        for line in lines:
            if line[-1]!='\n':
                file.write(line+'\n')
            else:
                file.write(line)

In [3]:
os.chdir(largedir)
to_do = os.listdir()
os.chdir(initdir)
print(to_do.__len__())
to_do = list(estb_src(set(to_do)))
print(to_do.__len__())

53
22


In [85]:
def all_cond(line,xs):
    res = None
    if all_.match(line):
        for targ in re.split(w_out_space_divide,re.split(all_divide,line[:-1])[-1]):
#             print(targ)
            xs[targ] = {} 
        xs['text'].append(line)
        res = xs
    return res
def cmp_cond(line,xs):
    res = None
    if comp_.match(line):
        cmp = re.split(w_out_space_divide,line)[1:]
        if cmp[0] == 'g++':
            cmp[0] = 'clang++'
        else:
            cmp[0] = 'clang'
        
        tmp = [cmp[0],clang_flags,'-o']+cmp[cmp.index('-o')+2:]

        
        targ = cmp[cmp.index('-o')+1]
        cmp = cmp[1:cmp.index('-o')]
        for arg in cmp:
            xs[targ][re.split(cf_reg,arg)[0]] = {'cmp':tmp,'opt':None,'abs_name':re.split(cf_reg,arg)[0],'c_name':arg}
#         xs['text'].append('\t\t'+' '.join(tmp)+'\n')
#         print('\t\t'+' '.join(tmp)+'\n')
        res = xs
    return res

def name_cond(line,xs):
    res = None
    if reg_name.match(line):
#         print(line)
        res = xs
    return res

In [67]:
# os.chdir(initdir)
# pickle.dump(todo,open('comp1.init','wb'))

In [186]:
os.chdir(initdir)
recover_make(largedir,pickle.load(open('comp1.init','rb')))

0

In [ ]:
# todo = {} 
# for line,signa in select_groups(to_do,largedir)['comp']:
#     todo[signa] = {'text':[],'exec':[],'target':{},'opts':[]}
# make_routine(largedir,todo,[all_cond,cmp_cond,name_cond])

In [155]:
def check_existance(xs,exs,_str):
    flag = False
    len_ = exs.__len__()
    exs.add(_str)
    if  len_!=exs.__len__():
        xs['text'].append(_str)
        flag = True
    else:
        print('asd')

    return flag
    

In [187]:
todo = {} 
for line,signa in select_groups(to_do,largedir)['comp']:
    todo[signa] = {'text':[],'opts':[]}
make_routine(largedir,todo,[all_cond,cmp_cond,name_cond])

for ult in todo:
    _all = [todo[ult]['text'][0][:-1]]
    arg_gp = list(filter(lambda x: (x!='text') and (x!='opts'),list(todo[ult])))
    
    for targ in arg_gp:
        for arg in todo[ult][targ]:
            name_ = todo[ult][targ][arg]['abs_name']
            _all+=[name_+'.ll',name_+'.bc']
    _all = ' '.join(_all) +'\n'

    todo[ult]['text'][0] = _all
    todo[ult]['text'].insert(0,phony_routine)
    
    set_tmp = set()
    for targ in arg_gp:
        for arg in todo[ult][targ]:
            c_name = todo[ult][targ][arg]['c_name']
            a_name = todo[ult][targ][arg]['abs_name']
            conf = todo[ult][targ][arg]['cmp']

            check_existance(todo[ult],set_tmp,('{}: {}\n'.format(a_name+'.ll',c_name)))
            
            check_existance(todo[ult],set_tmp,'\t\t{0} {1} -o {2} {3}\n'.format(' '.join(conf[:conf.index('-o')])
                                                          ,c_name,a_name+'.ll'
                                                             ,' '.join(conf[conf.index('-o')+1:])))
            


            if check_existance(todo[ult],set_tmp,mopt_routine(a_name+'.ll',(a_name+'.bc'))):
                todo[ult]['opts'].append(todo[ult]['text'].__len__()-1)
    
    for targ in arg_gp:
#         print(' '.join(conf[:1]+conf[2:conf.index('-o')]))
        targ_bc = ' '.join(list(map(lambda x:x+'.bc',list(todo[ult][targ]))))
        todo[ult]['text'].append('{0}:{1}\n\t\t{2} {3} -o {4} {5}\n'.format(targ
                                               ,targ_bc,' '.join(conf[:1]+conf[2:conf.index('-o')])
                                               ,targ_bc,targ
                                               ,' '.join(conf[conf.index('-o')+1:])))
#         print(todo[ult]['text'][-1])
    todo[ult]['text'].append(clean_routine(re.split(all_divide,_all)[1]))
#     todo[ult]['text'] = list(set(todo[ult]['text']))

asd
asd
asd
asd
asd
asd


In [188]:
err = set()
res_opt = {'-O0':{},'-O1':{},'-O2':{},'-O3':{},'-Os':{},'-Oz':{}}
for ult in todo:
    os.chdir(absdir+'unix_cmp')
    dir_ = make_dir("{}/{}/".format(ult,'source'))[0]
    os.chdir(dir_)
    for option in opt_:
        for op in todo[ult]['opts']:  
            _opt = (todo[ult]['text'][op])

            tmp = _opt.split('\n')[1]
            tmp = re.split(w_out_space_divide,tmp)
            tmp[2] = option
            todo[ult]['text'][op] = '\n'.join([_opt.split('\n')[0],'\t\t'+' '.join(tmp)+'\n'])
        with open('Makefile','wt') as make_file:
            for line in todo[ult]['text']:
                make_file.write(line)    
        make_proc(['make'],log_file,ult)
        res = -1
        try:
            for targ in list(filter(lambda x: (x!='text') and (x!='opts'),list(todo[ult]))):
                res =subprocess.check_output("wc -c {}".format(targ), stderr=subprocess.STDOUT,
                                                       shell=True)
                res_opt[option][ult] = res
                print(res)
        except Exception:
            err.add(ult)
            print(ult,targ)

        make_proc(['make','clean'],log_file,ult)


    


b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'37024 main\n'
b'41040 main\n'
b'41040 main\n'
b'41040 main\n'
b'41040 main\n'
b'41040 main\n'
b'34920 test\n'
b'35024 test\n'
b'35024 test\n'
b'35024 test\n'
b'35024 test\n'
b'35024 test\n'
b'18104 bwtcoder\n'
b'22200 bwtcoder\n'
b'26296 bwtcoder\n'
b'26296 bwtcoder\n'
b'26296 bwtcoder\n'
b'22200 bwtcoder\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'13760 main\n'
b'13760 main\n'
b'13760 main\n'
b'13760 main\n'
b'13760 main\n'
b'13760 main\n'
b'40224 main\n'
b'40232 main\n'
b'40232 main\n'
b'40232 main\n'
b'40232 main\n'
b'40232 main\n'
b'13928 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13928 SMLMAIN\n'
b'34400 RopeStrings\n'
b'34448 RopeStrings\n'
b'34448 RopeStrings\n'
b'34448 RopeStrings\n'
b'34448 RopeStrings\n'
b'30352 RopeStrings\n'
b'24112 main\n'
b'24168 main\n'
b'24168 main\n'
b'24

In [177]:
res_opt

{'-O0': {'Huffman compression algorithm.': b'20672 hdecomp\n'},
 '-O1': {'Huffman compression algorithm.': b'20440 hdecomp\n'},
 '-O2': {'Huffman compression algorithm.': b'20440 hdecomp\n'},
 '-O3': {'Huffman compression algorithm.': b'20440 hdecomp\n'},
 '-Os': {'Huffman compression algorithm.': b'20440 hdecomp\n'},
 '-Oz': {'Huffman compression algorithm.': b'20440 hdecomp\n'}}

In [78]:
# os.chdir(initdir+'result/')
# pickle.dump(res_opt,open('complex_file.result','wb'))